Todo:

- udělat to jako skript, co bere vše neoscrapované

Sólo sešit:
- mazat oscrapované soubory

In [92]:
import os
import re
from datetime import datetime
import pandas as pd

In [93]:
with open(os.path.join("downloads/2024-11-01","cd_Brno_Hamburg_D00_2024-11-01_00-32-19.txt"), "r", encoding="utf-8") as file:
    raw_html = file.read()
spojeni = raw_html.split("""<article""")
spojeni = spojeni[1:]
with open("pokus.txt", "w+", encoding="utf-8") as pokus_file:
    pokus_file.write("\n".join(spojeni))

In [97]:
def oscrapuj_cd(slozka, soubor):
    
    with open(os.path.join(slozka, soubor), "r", encoding="utf-8") as file:
        raw_html = file.read()
    spojeni = raw_html.split("""<article""")
    spojeni = spojeni[1:]
    slovniky = []
    for s in spojeni:
        s = s.splitlines()
        slovnik = {}
        slovnik['prodejce'] = "ČD"
        slovnik['vlaky'], slovnik['obsazenost'], slovnik['zpozdeni'] = [], [], []
        oscrapovano = re.search(r'\d{4}-\d{2}-\d{2}_\d{2}-\d{2}-\d{2}', soubor).group(0)
        slovnik['oscrapovano'] = datetime.strptime(oscrapovano, "%Y-%m-%d_%H-%M-%S")
        den = False
        for radek in s:
            if "text: to(lastTrain($index()))" in radek:
                slovnik['kam'] = radek.split('>')[-2].split('<')[0]
            if "text: from(firstTrain($index()))" in radek:
                slovnik['odkud'] = radek.split('>')[-2].split('<')[0]
            if ("text: journeyDateTextFrom()" in radek) and (den == False):
                den = radek.split('>')[-2].split('<')[0]
            if "text: depTime(firstTrain($index()))" in radek:
                cas_odjezdu = re.search(r"\d{1,2}:\d\d", radek).group(0)
            if "text: buyButtonText()" in radek:
                cena = radek.split('-->')[1].split('<!--')[0].replace(" Kč","")
                try:
                    slovnik['cena'] = int(cena)
                except:
                    pass
            if "Zjistit cenu" in radek:
                slovnik['cena_poznamka'] = "Zjistit cenu"
            if "Cena v dalším kroku" in radek:
                slovnik['cena_poznamka'] = "Cena v dalším kroku"
            if "text: model.trainTypeAndNum" in radek:
                if radek.split('-->')[1].split('<!--')[0] not in slovnik['vlaky']:
                    slovnik['vlaky'].append(radek.split('-->')[1].split('<!--')[0])
            if """<span aria-hidden="true" data-bind="visible: occupancyLevelFull(), text: occupancyLevelText()" style="display: none;">""" in radek:
                slovnik['obsazenost'].append(radek.split('>')[-2].split('<')[0])
            if """<span data-bind="text: timeLength, visible: timeLength != null &amp;&amp; timeLength != ''">""" in radek:
                jizdni_doba = radek.split('>')[-2].split('<')[0].split(":")
                slovnik['jizdni_doba'] = (int(jizdni_doba[0]) * 60) + int(re.search(r'\d\d', jizdni_doba[1]).group(0))
            if """<span data-bind="text: distance, visible: distance != null &amp;&amp; distance != ''" class="mobile-hidden">""" in radek:
                vzdalenost = radek.split('>')[-2].split('<')[0].replace(" km","")
                try:
                    slovnik['vzdalenost'] = int(vzdalenost)
                except:
                    pass
            if """<span class="icon icon-bus" data-bind="ifnot: isLegend, visible: !isLegend &amp;&amp; icoSrc, css: icoSrc, attr: {title: desc, 'aria-label': desc}" title="Náhradní autobusová doprava" aria-label="Náhradní autobusová doprava"></span>""" in radek:
                slovnik['nahradni_bus'] = True
#            if """OMEZENÍ NA TRASE""" in radek:
#                slovnik['omezeni'] = True
#            if """BEZ OMEZENÍ NA TRASE""" in radek:
#                slovnik['omezeni'] = False
            if """<span aria-hidden="true" data-bind="text: delayText()">""" in radek:
                if "+" in radek:
                    try:
                        slovnik['zpozdeni'].append(int(radek.split('+')[1].strip().split(' ')[0]))
                    except:
                        slovnik['zpozdeni'].append('chyba')
                elif "><" in radek:
                    slovnik['zpozdeni'].append(0)
            if "Místenka zdarma" in radek:
                slovnik['mistenka_zdarma'] = True
        slovnik['den'] = den.split(' ')[0].lower().strip()
        slovnik['odjezd'] = datetime.strptime('-'.join(den.split(' ')[1].split('.')[::-1]) + " " + cas_odjezdu, "%Y-%m-%d %H:%M")
        slovnik['predstih'] = slovnik['odjezd'] - slovnik['oscrapovano']
        if len(slovnik['vlaky']) == 1:
            slovnik['vlaky'] = [slovnik['vlaky'][0]]
            try:
                slovnik['obsazenost'] = [slovnik['obsazenost'][0]] ## todo: obsazenost se přidává n+1×, toto je laciný workaround pro většinu případů 
            except:
                pass
        slovnik['zpozdeni'] = slovnik['zpozdeni'][1::2]
        slovnik['prestupy'] = len(slovnik['vlaky']) - 1
        slovnik['prostredek'] = 'vlak'
        slovniky.append(slovnik)
    return slovniky

In [99]:
for x in os.listdir("downloads"):
    kam = "data" 
    os.makedirs(kam, exist_ok=True)
    den = []
    cd = [y for y in os.listdir(f"downloads/{x}") if "cd_" in y] 
    print(f"{x}: {len(cd)}")
    for y in cd:
        print(y)
        den = den + oscrapuj_cd(f"downloads/{x}",y)
    nazev_souboru = "cd_" + x + ".parquet"
    df_den = pd.DataFrame(den).sort_values(by="oscrapovano").reset_index(drop=True)
    df_den.to_parquet(os.path.join(kam,nazev_souboru))

2024-10-31: 61
cd_Brno_Hamburg_D00_2024-10-31_23-32-19.txt
cd_Brno_Hamburg_D01_2024-10-31_23-35-33.txt
cd_Brno_Hamburg_D02_2024-10-31_23-39-05.txt
cd_Brno_Hamburg_D03_2024-10-31_23-42-37.txt
cd_Brno_Hamburg_D04_2024-10-31_23-46-17.txt
cd_Brno_Hamburg_D05_2024-10-31_23-50-13.txt
cd_Brno_Hamburg_D06_2024-10-31_23-54-11.txt
cd_Brno_Hamburg_D07_2024-10-31_23-58-19.txt
cd_Brno_Praha_D00_2024-10-31_23-30-46.txt
cd_Brno_Praha_D01_2024-10-31_23-33-55.txt
cd_Brno_Praha_D02_2024-10-31_23-37-22.txt
cd_Brno_Praha_D03_2024-10-31_23-40-51.txt
cd_Brno_Praha_D04_2024-10-31_23-44-26.txt
cd_Brno_Praha_D05_2024-10-31_23-48-16.txt
cd_Brno_Praha_D06_2024-10-31_23-52-12.txt
cd_Brno_Praha_D07_2024-10-31_23-56-13.txt
cd_Liberec_Salzburg_D00_2024-10-31_23-32-44.txt
cd_Liberec_Salzburg_D01_2024-10-31_23-36-00.txt
cd_Liberec_Salzburg_D02_2024-10-31_23-39-31.txt
cd_Liberec_Salzburg_D03_2024-10-31_23-43-06.txt
cd_Liberec_Salzburg_D04_2024-10-31_23-46-47.txt
cd_Liberec_Salzburg_D05_2024-10-31_23-50-44.txt
cd_Libere